In [ ]:
import pandas as pd 
import numpy as np
import statsmodels.api as sm
import scipy.linalg

In [ ]:
df = pd.read_stata('cornwell.dta')
df = df.assign(cte = 1)
df = df[['county', 'year', 'lcrmrte','lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']].assign(cte = 1)
df = df.dropna()
df = pd.concat([df, pd.get_dummies(df['year'])], axis = 1 )
x = df[['cte', 'lprbarr', 'lprbconv', 'lprbpris', 'lavgsen', 'lpolpc']]
x = pd.concat([x, pd.get_dummies(df['year'])], axis = 1 )
y = df['lcrmrte']

OLS COMÚN

In [ ]:
pols = sm.OLS(y,x).fit()
pols.summary()


Errores estandar robustos a la presencia de heterocedasticidad

In [ ]:
pols_rob = sm.OLS(y,x).fit(cov_type='HC3')
pols_rob.summary()

## Ejercicio Número 3

In [12]:
np.random.seed(1313)
n = 5 
T = 2
omg = np.array([[1,0], [0,4]])
muestras = 1000
data = {'i': [], 't': [], 'y': [], 'cte':[] , 'x': [], 'u': []}


for i in range(1,n+1) : 
    for t in range(1,T+1) : 
        data['cte'].append(1)
        data['i'].append(i)
        data['t'].append(t)
        x = np.random.uniform(1,20,1)
        x = x.astype(float)
        u = np.random.normal(0,omg.item(t-1,t-1))
        data['x'].append(x)
        data['u'].append(u)
        data['y'].append(1 + x + u)


data = pd.DataFrame(data=data)
y = data['y'].astype(float)
x = data[['cte', 'x']].astype(float)

In [13]:
reg_ols = sm.OLS(y,x).fit()

In [14]:
u_hat = {'i': data['i'], 't': data['t'], 'u_jt': reg_ols.resid.tolist()}
u_hat = pd.DataFrame(data = u_hat)
u_hat

,i,t,u_jt
0,1,1,-2.619755
1,1,2,-0.951289
2,2,1,-0.427754
3,2,2,-0.389934
4,3,1,-0.548701
5,3,2,1.604623
6,4,1,-0.465574
7,4,2,4.475627
8,5,1,-0.132044
9,5,2,-0.545199


In [16]:
omg_est = np.array([[0,0], [0,0]])
for l in range(0,n*T-1,2): 
    u_j = np.array([[u_hat['u_jt'][l]], [u_hat['u_jt'][l+1]]])
    omg_est = omg_est + u_j.dot(u_j.T)
omg_est

array([[ 7.5813567 , -0.23326474],
       [-0.23326474, 23.96028772]])

In [ ]:
#x = x.to_numpy()
#y = y.to_numpy()
x_j = np.asanyarray([[x[i], x[i+1]] for i in range(0,n*T,2)])
y_j = np.asanyarray([[y[i], y[i+1]] for i in range(0,n*T,2)])
var_est = np.zeros((2,2), float) #KXK cambiar nombre esta en realidad es la inversa
xT_omg_y = np.zeros((2,1), float) #kx1
for i in range(0,n): 
    temp = x_j[i].T.dot(omg_inv).dot(x_j[i])
    var_est = var_est + temp
    temp2 = x_j[i].T.dot(omg_inv).dot(y_j[i])
    xT_omg_y = xT_omg_y + temp2
xT_omg_y = xT_omg_y[1] #Me quedo solo con el primer vector porque numpy hace broadcasting 
beta_fgls = np.linalg.inv(var_est).dot(xT_omg_y)
beta_fgls #No son muy diferentes a los ols, me gusta 

In [17]:
b = omg_est.reshape(1,T,T).repeat((n*T/2),axis=0)
prueba = scipy.linalg.block_diag(*b)
prueba = pd.DataFrame(prueba)
#y = data['y'].astype(float)
#x = data[['cte', 'x']].astype(float)
fgls = sm.GLS(y,x,prueba).fit()
fgls.summary()

/home/creambbq/anaconda3/lib/python3.9/site-packages/scipy/stats/stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=10
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


<class 'statsmodels.iolib.summary.Summary'>
"""
                            GLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.936
Model:                            GLS   Adj. R-squared:                  0.928
Method:                 Least Squares   F-statistic:                     117.4
Date:                Sat, 16 Apr 2022   Prob (F-statistic):           4.65e-06
Time:                        19:09:28   Log-Likelihood:                -18.555
No. Observations:                  10   AIC:                             41.11
Df Residuals:                       8   BIC:                             41.71
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
cte            2.4684      1.261      1.958      0.086      -0.439       5.376
x              0.9312      0.086     10.834      0.000       0.733       1.129
==============================================================================
Omnibus:                        5.452   Durbin-Watson:                   1.645
Prob(Omnibus):                  0.065   Jarque-Bera (JB):                1.610
Skew:                           0.766   Prob(JB):                        0.447
Kurtosis:                       4.232   Cond. No.                         37.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [ ]:
test = np.array([[31.24440301, -0.95044039,0.,0.,0.,0.,0.,0.,0.,0.], 
         [-0.95044039, 24.67852892,0.,0.,0.,0.,0.,0.,0.,0.],
         [0.,0.,31.24440301, -0.95044039,0.,0.,0.,0.,0.,0.], 
         [0.,0.,-0.95044039, 24.67852892,0.,0.,0.,0.,0.,0.],
         [0.,0.,0.,0.,31.24440301, -0.95044039,0.,0.,0.,0.], 
         [0.,0.,0.,0.,-0.95044039, 24.67852892,0.,0.,0.,0.],
         [0.,0.,0.,0.,0.,0.,31.24440301, -0.95044039,0.,0.], 
         [0.,0.,0.,0.,0.,0.,-0.95044039, 24.67852892,0.,0.],
         [0.,0.,0.,0.,0.,0.,0.,0.,31.24440301, -0.95044039], 
         [0.,0.,0.,0.,0.,0.,0.,0.,-0.95044039, 24.67852892]])
test = pd.DataFrame(test)

In [ ]:
y = data['y'].astype(float)
x = data[['cte', 'x']].astype(float)

In [ ]:
fgls = sm.GLS(y,x,test).fit()
fgls.summary()

In [45]:
beta_fgls

array([2.25092463, 0.98571238])